In [ ]:
import re
import pandas as pd
import requests

import time #modulo para controlar los tiempos y los sleep (evitar el error 429)

#DEFINO LOS HEADERS: 
headers = {
"X-RapidAPI-Key": "28a3c3ab32msha5ca6e00170b6c7p197d01jsn44c7f725210a",
"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

#DEFINO LOS GENEROS 
genres = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]

sleep_time = 5

#SACAR TODAS LAS URLS: 

for genre in genres:
  
    for year in range(1990, 2023):
        page = 1
        #url = f"https://moviesdatabase.p.rapidapi.com/titles?year={year}&genre={genre}&page={page}"
        while True:  
        #for page in range(1, 11):
            url = f"https://moviesdatabase.p.rapidapi.com/titles?year={year}&genre={genre}&page={page}"
            sleep_time
            response = requests.get(url, headers=headers)
            #SI LA RESPUESTA ES 200 ENTONCES COMIENZO A HACER TODO EL LOOP DE LINKS Y EXTRAER DATOS: 
            if response.status_code == 200:
                # Procesamos la respuesta da data aqui, lo convertimos a JSONy extraemos la información:
                datos = response.json()
                lista_de_diccionarios = datos["results"]
                lista_peliculas = []

                for datos in lista_de_diccionarios: 
                    
                    #lista que usamos para ir appendeando toda la información de abajo /cada peli y será una lista "de paso"
                    lista_consumible = []
                    # aqui creamos las variables con las "rutas " que hemos ido buscando arriba /dentro del json 
                    tipo = datos["titleType"]["text"]
                    texto = datos["titleText"]["text"]
                    #para que no me de error el NONE de aquellas pelis que no tengan mes o año. añado el check de: 
                    release_date = datos.get("releaseDate")
                    if release_date:
                        mes = release_date.get("month")
                        anio = release_date.get("year")
                    else:
                        mes = None
                        anio = None
                    #hasta aqui el control del mes y año 
                    id = datos["id"]

                    if tipo == "Movie" or tipo =="Short":
                        lista_consumible.append(tipo)
                        lista_consumible.append(texto)
                        lista_consumible.append(mes)
                        lista_consumible.append(anio)
                        lista_consumible.append(id)
                        sleep_time

                    #Si la var está, entonces lo appendeamos a la tupla. Tiene q estar aqui dentro porque si esta fuera, irá creando tuplas vacías. 
                    lista_peliculas.append(tuple(lista_consumible))
                print(f"Received data for {genre} in {year}, page {page}")


                page = page + 1
                   

               
            #SI LA RESPUESTA NO ES 200. TELL ME SOMETHING I DONT KNOW
            elif response.status_code == 429:
                print("Too many requests. Sleeping...")

            # print(response.text) 
            # time.sleep(sleep_time)

            else: # Otro error que no sea el 429 
                if "error" in response.text:
                    error_message = response.json().get("error")
                    print(f"Error Message: {error_message}")

            print("...")


   

In [ ]:
lista_peliculas

In [ ]:
df_peliculas = pd.DataFrame(lista_peliculas)

In [ ]:
df_peliculas

In [ ]:
df_peliculas.to_csv("datos_api.csv")

In [ ]:
df_peliculas.to_excel("datos_api.xlsx")

In [ ]:
df_peliculas

In [ ]:
import re
import pandas as pd
import requests
import time

# Define the headers for the API
headers = {
    "X-RapidAPI-Key": "28a3c3ab32msha5ca6e00170b6c7p197d01jsn44c7f725210a",
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

# Define the list of movie genres
genres = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]

sleep_time = 5

# Iterate through genres, years, and pages
for genre in genres:
    for year in range(1990, 2023):
        next_page = True
        while next_page:
            url = f"https://moviesdatabase.p.rapidapi.com/titles?year={year}&genre={genre}"
            sleep_time
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                datos = response.json()
                lista_de_diccionarios = datos["results"]
                lista_peliculas = []

                for datos in lista_de_diccionarios:
                    lista_consumible = []
                    tipo = datos["titleType"]["text"]
                    texto = datos["titleText"]["text"]
                    release_date = datos.get("releaseDate")
                    if release_date:
                        mes = release_date.get("month")
                        anio = release_date.get("year")
                    else:
                        mes = None
                        anio = None
                    id = datos["id"]

                    if tipo == "Movie" or tipo == "Short":
                        lista_consumible.append(tipo)
                        lista_consumible.append(texto)
                        lista_consumible.append(mes)
                        lista_consumible.append(anio)
                        lista_consumible.append(id)

                        lista_peliculas.append(tuple(lista_consumible))

                next_page = datos.get("nextPage")
                if next_page:
                    url = f"{url}&page={next_page}"  # Update the URL to fetch the next page
                time.sleep(sleep_time)
            else:
                print(f"Failed to retrieve data for {genre} in {year}. Status Code: {response.status_code}")
                if "error" in response.text:
                    error_message = response.json().get("error")
                    print(f"Error Message: {error_message}")

            print("...")
